In [1]:
import os
import time
import sys
import h5py
import re
import tensorflow         as tf
import numpy              as np
import pandas             as pd
from matplotlib           import pyplot as plt 
from os                   import path
from tqdm                 import tqdm

print("========================================================")
print(" TensorFlow version: {}".format(tf.__version__))
print(" Eager execution: {}".format(tf.executing_eagerly()))

 TensorFlow version: 2.5.0
 Eager execution: True


In [2]:
WORKSPACE_PATH      = os.environ['WORKSPACE_PATH']
SurQCTFldr          = WORKSPACE_PATH + '/SurQCT/surqct/'

RatesType           = 'KExcit'
ExcitType           = 'KInel'
NNRunIdx            = 12

PathToRunFld        = SurQCTFldr + '/../' + RatesType + '/all_temperatures/' + ExcitType + '/' 

TTran               = [1500.0]
ReadPlato           = True

MinRate             = 1.e-15

iIdxVec             = np.array([2, 3, 4, 5], dtype=int)

In [3]:
print("\n[SurQCT]: Loading Modules and Functions ...")

sys.path.insert(0, SurQCTFldr  + '/src/Reading/')
from Reading  import read_levelsdata, read_diatdata, read_groupsmapping, read_sampledinitiallevels

InputFld = PathToRunFld + '/Run_' + str(NNRunIdx) + '/'
print(InputFld)
sys.path.insert(0, InputFld)


[SurQCT]: Loading Modules and Functions ...
/home/maitreyee/WORKSPACE//SurQCT/surqct//../KExcit/all_temperatures/KInel//Run_12/


In [4]:
print("\n[SurQCT]: Keep Loading Modules and Functions...")
from SurQCT_Input import inputdata

print("\n[SurQCT]: Initializing Input ...")
InputData    = inputdata(WORKSPACE_PATH, SurQCTFldr)

Prefix                    = 'Run_'
InputData.NNRunIdx        = NNRunIdx
InputData.PathToRunFld    = InputData.PathToRunFld+'/'+Prefix+str(InputData.NNRunIdx)
InputData.PathToFigFld    = InputData.PathToRunFld+'/'+InputData.PathToFigFld
InputData.PathToParamsFld = InputData.PathToRunFld+'/'+InputData.PathToParamsFld
InputData.PathToDataFld   = InputData.PathToRunFld+'/Data/'                                                               
InputData.PathToParamsFld = InputData.PathToRunFld+'/Params/' 
InputData.KineticFldr     = InputData.WORKSPACE_PATH+'/Air_Database/Run_0D_surQCT/database/kinetics/O3_UMN_Run'+str(NNRunIdx)+'/'


[SurQCT]: Keep Loading Modules and Functions...

[SurQCT]: Initializing Input ...


In [5]:
print("\n[SurQCT]: Loading Final Modules ... ")

sys.path.insert(0, SurQCTFldr  + '/src/Model/' + InputData.ApproxModel + '/')
from Model import model


InputData.DefineModelIntFlg  = 0
InputData.TrainIntFlg        = 0
InputData.TransferFlg        = False
NN_KExcit                    = model(InputData, InputData.PathToRunFld, None, None)
NN_KExcit.load_params(InputData.PathToParamsFld)


[SurQCT]: Loading Final Modules ... 
[SurQCT]:   Variables Selected for Training:
[SurQCT]:     x =  ['EVib_i', 'ERot_i', 'rMin_i', 'rMax_i', 'VMin_i', 'VMax_i', 'Tau_i', 'ri_i', 'ro_i', 'TTran_i', 'EVib_Delta', 'ERot_Delta', 'rMin_Delta', 'rMax_Delta', 'VMin_Delta', 'VMax_Delta', 'Tau_Delta', 'ri_Delta', 'ro_Delta', 'TTran_Delta']
[SurQCT]:     y =  KExcit
[SurQCT]:   Loading ML Model from Folder: /home/maitreyee/WORKSPACE//SurQCT/surqct//../KExcit/all_temperatures/KInel//Run_12/MyModel
[SurQCT]:   Restoring ML Model Weigths to File: /home/maitreyee/WORKSPACE//SurQCT/surqct//../KExcit/all_temperatures/KInel//Run_12/MyModel/MyWeights.h5
[SurQCT]:   Loading ML Model Parameters from File:  /home/maitreyee/WORKSPACE//SurQCT/surqct//../KExcit/all_temperatures/KInel//Run_12/Params//ModelCheckpoint/cp-8534.ckpt


In [6]:
OtherVar           = InputData.OtherVar
xVarsVec_i         = InputData.xVarsVec_i + ['vqn','jqn']
xVarsVec_Delta     = InputData.xVarsVec_Delta
xVarsVec           = list(set(xVarsVec_i) | set(xVarsVec_Delta))
print('[SurQCT]:   Reading Variables: ', xVarsVec)

InputData.MultFact = 1.e+9
MinValueTrain      = 1.e-16 * InputData.MultFact
MinValueTest       = 1.e-16 * InputData.MultFact
NoiseSD            = 1.e-13 * InputData.MultFact

NMolecules         = len(InputData.PathToLevelsFile)

[SurQCT]:   Reading Variables:  ['VMax', 'rMin', 'ERot', 'Tau', 'rMax', 'ro', 'jqn', 'EVib', 'VMin', 'vqn', 'ri']


In [27]:
LevelsData = []
DiatData   = []
NLevels    = []
for iMol in range(NMolecules):

    LevelsDataTemp = read_levelsdata(InputData.PathToLevelsFile[iMol], xVarsVec, '')
    LevelsData.append(LevelsDataTemp)

    DiatDataTemp = read_diatdata(InputData.PathToDiatFile[iMol], InputData.Molecules[iMol], np.array([TTran]), np.array([TTran]))
    DiatData.append(DiatDataTemp)
    
    NLevelsTemp    = LevelsDataTemp.shape[0]
    NLevels.append(NLevelsTemp)
    
GroupMap = read_groupsmapping(InputData.PathToGrouping)

[SurQCT]:   Reading Molecular Levels Data from: /home/maitreyee/WORKSPACE//Air_Database/Run_0D/database/levels/46DPM_Sampled/O2_LogELogR.csv
[SurQCT]:   Reading Molecular Levels Data from: /home/maitreyee/WORKSPACE//CoarseAIR/coarseair/dtb/Molecules/O2/UMN/FromUMN_Sorted.inp
[SurQCT]:   Reading Molecular Levels Data from: /home/maitreyee/WORKSPACE//Air_Database/Run_0D/database/levels/46DPM_Sampled/O2_LogELogR.csv
[SurQCT]:   Reading Molecular Levels Data from: /home/maitreyee/WORKSPACE//CoarseAIR/coarseair/dtb/Molecules/O2/UMN/FromUMN_Sorted.inp
[SurQCT]:       Reading Grouping from File: /home/maitreyee/WORKSPACE//Air_Database/Run_0D/database/grouping/O3_UMN/O2/LevelsMap_DPM45.csv


In [38]:
#========================================================================================================
def read_kexcitdata(InputData, PathToHDF5File, TTra, TInt, NProcTypes):
    print('[SurQCT]:       Reading HDF5 File from: ' + PathToHDF5File + ' for Excitation Rates at Temperature ' + str(int(TTra)) + 'K')

    HDF5Exist_Flg = path.exists(PathToHDF5File)
    if (HDF5Exist_Flg):
        f = h5py.File(PathToHDF5File, 'a')
    else:
        f = {'key': 'value'}

    TStr = 'T_' + str(int(TTra)) + '_' + str(int(TInt)) + '/Rates/'
    grp  = f[TStr]

    Data  = grp["Inel"]
    KInel = Data[...]

    KOther = []
    for iProc in range(2, NProcTypes):
        ExchStr    = "Exch_" + str(iProc-1)
        Data       = grp[ExchStr] 
        KOtherTemp = Data[...]
        KOther.append(KOtherTemp)

    f.close()   


    return KInel, KOther

#========================================================================================================
def read_kexcitdata_PLATO(InputData, PathToKineticFldr, TTra):
    print(PathToKineticFldr)
    PathToFile    = PathToKineticFldr + '/T' + str(int(TTra)) + 'K/Inel.dat'
    Rates = np.zeros((NLevels[0],NLevels[0]))
    with open(PathToFile) as f:
        for lines in f:
            lines = lines.replace('(',',')
            lines = lines.replace(')',',')
            lines = lines.replace('+',',')
            linestemp = re.split('O2|O|=|:|,',lines);
            iIdx    = int(linestemp[2]);
            jIdx    = int(linestemp[8]);
            RateTemp = float(linestemp[12]);
            Rates[iIdx-1,jIdx-1] = RateTemp;
    f.close()
    
    return Rates

#========================================================================================================
def Save_RatesAtT_HDF5( NNRates, PathToHDF5File, TTra, TInt ):
    PathToFile    = PathToHDF5File
    HDF5Exist_Flg = path.exists(PathToFile)
    f = h5py.File(PathToFile, 'a')
    
    TStr = 'RunI' + str(int(NNRunIdx)) + '/T_' + str(int(TTra)) + '_' + str(int(TInt)) + '/Rates/'       
    if TStr in f.keys():
        grp       = f[TStr]
        Data      = grp["Inel"]
        Data[...] = NNRates

    else:
        grp       = f.create_group(TStr)
        Proc1     = grp.create_dataset("Inel", data=NNRates, compression="gzip", compression_opts=9)

    f.close()
    
#========================================================================================================
def compute_BWDRates(KInel_FWD, TTra):
    Str = 'q_'+str(int(TTra))
    KInel_NN = KInel_FWD
    for iLevel in tqdm(range(NLevels[0]), desc='[SurQCT]:     Computing backward rates'):
        for jLevel in range(NLevels[0]):
            if (DiatData[1]['EInt'].to_numpy()[jLevel] > DiatData[0]['EInt'].to_numpy()[iLevel]):
                KInel_NN[iLevel, jLevel] = KInel_FWD[jLevel, iLevel] * (DiatData[1][Str].to_numpy()[jLevel]) / (DiatData[1][Str].to_numpy()[iLevel])

    return KInel_NN

In [41]:
KInelMat, KExchMatList = read_kexcitdata(InputData, InputData.PathToHDF5File, TTran[0], TTran[0], 3)

if (ExcitType == 'KInel'):
    KExcit = KInelMat
elif (ExcitType == 'KExch'):
    KExcit = KExchMatList[0]
else:
    KExcit = KInelMat + KExchMatList[0]
    
if (ReadPlato):    
    KInel_NN_FWD = read_kexcitdata_PLATO(InputData,InputData.KineticFldr,TTran[0])
    
KInel_NN = compute_BWDRates(KInel_NN_FWD, TTran[0])
PathToNNHDF5File = WORKSPACE_PATH  + '/Air_Database/HDF5_Database_NN/O3_UMN.hdf5'
Save_RatesAtT_HDF5(KInel_NN, PathToNNHDF5File, TTran[0], TTran[0])
    

-5.112689786907744
/home/maitreyee/WORKSPACE//Air_Database/Run_0D_surQCT/database/kinetics/O3_UMN_Run12/
2 3 0.0 1.155e-10
2 4 0.0 5.8815e-11
2 5 0.0 3.6781e-11
3 4 0.0 1.2141e-10
3 5 0.0 6.0056e-11
4 5 0.0 1.2334e-10


In [54]:
KInelMat, KExchMatList = read_kexcitdata(InputData, InputData.PathToHDF5File, TTran[0], TTran[0], 3)

if (ExcitType == 'KInel'):
    KExcit = KInelMat
elif (ExcitType == 'KExch'):
    KExcit = KExchMatList[0]
else:
    KExcit = KInelMat + KExchMatList[0]
    

[SurQCT]:       Reading HDF5 File from: /home/maitreyee/WORKSPACE//Air_Database/HDF5_Database/O3_UMN.hdf5 for Excitation Rates at Temperature 1500K


In [52]:
KInel_NN = compute_BWDRates(KInel_NN_FWD, TTran[0])

[SurQCT]:     Computing backward rates: 100%|██████████| 6115/6115 [09:08<00:00, 11.15it/s]


/T_1500_1500


['Beautiful', 'is', 'better', 'than', 'ugly']
